$y = \displaystyle \operatorname{erf}(x) = \frac{2}{\sqrt{\pi}}\int_0^x e^{-t^2}\,dt$ の値域は $-1<y<1$.

これの逆函数が $x = \operatorname{erfiv}(y)$

In [1]:
using SymPy

# Override
# https://github.com/jverzani/SymPyCore.jl/blob/main/src/SymPy/show_sympy.jl#L31-L34
@eval SymPy begin
function Base.show(io::IO,  ::MIME"text/latex", x::SymbolicObject)
    out = _sympy_.latex(↓(x), mode="inline",fold_short_frac=false)
    out = replace(out, r"\\frac{"=>"\\dfrac{")
    print(io, string(out))
end
end

using Base.Math: @horner

In [2]:
"https://github.com/JuliaMath/SpecialFunctions.jl/blob/master/src/erf.jl#L322-L363"
function erfinv_Float32(x, xx) # xx = dummy of x
    a = abs(x)
    aa = abs(xx)
    if aa >= 1.0f0
        if xx == 1.0f0
            return Inf32
        elseif xx == -1.0f0
            return -Inf32
        end
        throw(DomainError(a, "`abs(x)` cannot be greater than 1."))
    elseif aa <= 0.75f0 # Table 10 in Blair et al.
        t = x*x - 0.5625f0
        return x * @horner(t, -0.13095_99674_22f2,
                               0.26785_22576_0f2,
                              -0.92890_57365f1) /
                   @horner(t, -0.12074_94262_97f2,
                               0.30960_61452_9f2,
                              -0.17149_97799_1f2,
                               0.1f1)
    elseif aa <= 0.9375f0 # Table 29 in Blair et al.
        t = x*x - 0.87890625f0
        return x * @horner(t, -0.12402_56522_1f0,
                               0.10688_05957_4f1,
                              -0.19594_55607_8f1,
                               0.42305_81357f0) /
                   @horner(t, -0.88276_97997f-1,
                               0.89007_43359f0,
                              -0.21757_03119_6f1,
                               0.1f1)
    else # Table 50 in Blair et al.
        t = inv(sqrt(-log1p(-a)))
        return @horner(t, 0.15504_70003_116f0,
                          0.13827_19649_631f1,
                          0.69096_93488_87f0,
                         -0.11280_81391_617f1,
                          0.68054_42468_25f0,
                         -0.16444_15679_1f0) /
              (copysign(t, x) *
               @horner(t, 0.15502_48498_22f0,
                          0.13852_28141_995f1,
                          0.1f1))
    end
end

erfinv_Float32

In [3]:
@syms x::positive
Base.copysign(t, x::Sym) = abs(t) * sign(x)

In [4]:
erfinv_Float32(x, 0.7)

           //                                     2\ / 2         \            
         x*\\32.0103198885918 - 9.28905773162842*x /*\x  - 0.5625/ - 13.095996
------------------------------------------------------------------------------
/ 2         \ //     2                   \ / 2         \                   \  
\x  - 0.5625/*\\1.0*x  - 17.7124786376953/*\x  - 0.5625/ + 30.9606151580811/ -

       \         
8566895/         
-----------------
                 
 12.0749425888062

In [5]:
erfinv_Float32(x, 0.9)

  // 2             \ //                   2                   \ / 2           
x*\\x  - 0.87890625/*\\0.423058122396469*x  - 2.33128403720912/*\x  - 0.878906
------------------------------------------------------------------------------
        / 2             \ //     2                   \ / 2             \      
        \x  - 0.87890625/*\\1.0*x  - 3.05460929870605/*\x  - 0.87890625/ + 0.8

  \                   \                    \
25/ + 1.06880593299866/ - 0.124025650322437/
--------------------------------------------
             \                              
9007431268692/ - 0.0882769823074341         

In [6]:
erfinv_Float32(x, 0.95)

/                                                                             
|                                                                             
|                                                                             
|                                                                             
|                                                           -1.1280814409256 +
|                                                                             
|                                                                             
|                                       0.690969347953796 + ------------------
|                                                                             
|                                                                             
|                    1.38271963596344 + --------------------------------------
|                                                                       ______
|                                                   

In [7]:
erfinv_Float32(-x, 0.95)

 /                                                                            
 |                                                                            
 |                                                                            
 |                                                                            
 |                                                           -1.1280814409256 
 |                                                                            
 |                                                                            
 |                                       0.690969347953796 + -----------------
 |                                                                            
 |                                                                            
 |                    1.38271963596344 + -------------------------------------
 |                                                                       _____
 |                                                  

In [8]:
using BenchmarkTools
using SpecialFunctions

In [9]:
@btime erfinv(0.7)
@btime erfinv(0.9)
@btime erfinv(0.95)

  1.000 ns (0 allocations: 0 bytes)
  1.100 ns (0 allocations: 0 bytes)
  1.200 ns (0 allocations: 0 bytes)


1.3859038243496802

In [10]:
@btime erfinv(0.7f0)
@btime erfinv(0.9f0)
@btime erfinv(0.95f0)

  1.100 ns (0 allocations: 0 bytes)
  1.200 ns (0 allocations: 0 bytes)
  1.100 ns (0 allocations: 0 bytes)


1.3859036f0

In [14]:
@benchmark randn(Float32)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  4.200 ns … 40.300 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.800 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.129 ns ±  1.418 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▁ █ ▇ ▆ ▃                                                
  ▂▃▁█▁█▁█▁█▁█▁█▅▁▄▁▄▁▃▁▃▁▄▁▅▁██▁█▁▆▁▄▁▃▁▂▁▂▁▂▂▁▂▁▂▁▂▁▂▁▂▁▂▂ ▃
  4.2 ns         Histogram: frequency by time         7.3 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [15]:
@benchmark randn()

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  3.400 ns … 34.000 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.900 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.200 ns ±  1.105 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

      ▂ █  █ ▃               ▁  ▂                             
  ▂▁▅▁█▁█▁▁█▁█▁▆▁▁▄▁▄▁▄▁▁▅▁▇▁█▁▁█▁▇▁▄▁▁▃▁▃▁▂▁▁▂▁▂▁▂▁▁▂▁▂▁▂▁▂ ▃
  3.4 ns         Histogram: frequency by time         5.9 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [32]:
quantile_stdnormal(p) = √2 * erfinv(2p - 1)
randn_Float32_inversemethod() = quantile_stdnormal(rand(Float32))
@benchmark randn_Float32_inversemethod()

BenchmarkTools.Trial: 10000 samples with 998 evaluations.
 Range (min … max):  14.429 ns … 73.447 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     16.733 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   17.753 ns ±  4.155 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▂▃█▅                                                      
  ▂▂▅████▇▅▃▃▂▃▃▃▃▄▄▃▂▂▂▂▂▂▂▂▁▂▂▂▁▂▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  14.4 ns         Histogram: frequency by time        37.7 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [31]:
@benchmark quantile_stdnormal(0.99)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  1.200 ns … 19.100 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.400 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.416 ns ±  0.386 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

                              █              ▄                
  ▅▁▁▁▁▁▁▁▁▁▁▁▁▁▇▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▂ ▂
  1.2 ns         Histogram: frequency by time         1.6 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [29]:
@benchmark rand(Float32)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  2.500 ns … 49.900 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.900 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.045 ns ±  1.205 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

     █                       ▁                                
  ▄▁▁█▁▁▄▁▁▄▁▁▄▁▁▃▁▁▃▁▁▄▁▁▅▁▁█▁▁▆▁▁▂▁▁▂▁▁▂▁▁▂▁▁▂▁▁▂▁▁▂▁▁▂▁▁▂ ▂
  2.5 ns         Histogram: frequency by time         4.4 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.